<a href="https://colab.research.google.com/github/MatteoOnger/VIA_Project/blob/main/VIA_EQM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **VIA Project: EqMotion Meets Assembly101 - Equivariant Motion Modeling for Manipulation Tasks**

*   **Author:** Matteo Onger
*   **Date:** September 2025

**Documentation**:
*   Dataset: [Assembly101](https://assembly-101.github.io/)
*   Model: [EqMotion](https://github.com/MediaBrain-SJTU/EqMotion)
*   Training Framework: [4DHands](https://github.com/FrancescoAgnelli3/4D_hands)

**Notes**:
*   For faster execution, please use a GPU-equipped runtime.

## Clone the Git Repo

In [ ]:
# clone training framework
!git clone https://github.com/FrancescoAgnelli3/4D_hands

## Load the dataset

### From SFTP server

In [4]:
from google.colab import userdata

In [5]:
# set the credentials and paths
local_path = '/content'
data_folder = 'data_our'
remote_path = '/datasets/AssemblyHands/assembly101-download-scripts/data_our'

host = userdata.get('phuse_server')     # server domain must be SET in SECRETS
user = userdata.get('phuse_user')       # server user must be SET in SECRETS
psw = userdata.get('phuse_psw')         # user password must be SET in SECRETS

user_at_host = user + '@' + host
data_path = local_path + '/' + data_folder

In [ ]:
# install sshpass
!sudo apt update
!sudo apt install sshpass

# create directories
!mkdir -p $remote_path
!mkdir -p ~/.ssh

# add server to known hosts
!ssh-keyscan $host >> ~/.ssh/known_hosts

# download the data
!sshpass -p $psw sftp -oBatchMode=no -b - $user_at_host <<< "get -r {remote_path} {local_path}"

### From drive

In [1]:
from google.colab import drive

In [ ]:
# mount Google Drive
drive.mount('/content/drive')

# set the paths
data_path = '/content/drive/MyDrive/Colab Notebooks/UNIMI/VIA Project/data_our'

## Run the models